In [ ]:
import random

# Constants
NUM_CITIES = 8
POPULATION_SIZE = 10  # Reduced population size
MAX_GENERATIONS = 5  # Reduced number of generations
MUTATION_PROBABILITY = 0.1

# Cities and their coordinates
cities = {
    'A': (60, 10),
    'B': (5, 90),
    'C': (50, 70),
    'D': (90, 40),
    'E': (30, 30),
    'F': (10, 50),
    'G': (40, 80),
    'H': (70, 20)
}

# Function to calculate the distance between two cities
def distance(city1, city2):
    x1, y1 = cities[city1]
    x2, y2 = cities[city2]
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

# Function to calculate the total distance of a path
def total_distance(path):
    total = 0
    for i in range(len(path) - 1):
        total += distance(path[i], path[i + 1])
    total += distance(path[-1], path[0])  # Return to the starting city
    return total

# Function to generate a random chromosome
def generate_chromosome(num_cities):
    return random.sample(list(cities.keys()), num_cities)

# Function to perform cyclic crossover
def cyclic_crossover(parent1, parent2):
    size = len(parent1)
    child1, child2 = [-1] * size, [-1] * size
    cycle = [0] * size

    while -1 in child1:
        index = child1.index(-1)
        cycle[index] = 1

        while cycle[index] == 1:
            child1[index] = parent1[index]
            child2[index] = parent2[index]
            index = parent1.index(parent2[index])

    return child1, child2

# Function to perform inversion mutation
def inversion_mutation(chromosome):
    start = random.randint(0, len(chromosome) - 1)
    end = random.randint(start, len(chromosome) - 1)
    chromosome[start:end+1] = reversed(chromosome[start:end+1])
    return chromosome

# Function to perform rank-based selection
def rank_based_selection(population, fitness_scores):
    sorted_population = [x for _, x in sorted(zip(fitness_scores, population))]
    rank_weights = [i / sum(range(len(population) + 1)) for i in range(1, len(population) + 1)]
    selected = random.choices(sorted_population, weights=rank_weights, k=2)
    return selected

# Function to perform genetic algorithm for TSP
def genetic_algorithm():
    population = [generate_chromosome(NUM_CITIES) for _ in range(POPULATION_SIZE)]

    for generation in range(MAX_GENERATIONS):
        fitness_scores = [total_distance(chromosome) for chromosome in population]
        sorted_indices = sorted(range(len(fitness_scores)), key=lambda k: fitness_scores[k])
        sorted_population = [population[i] for i in sorted_indices]

        selected_parents = rank_based_selection(sorted_population, fitness_scores)

        child1, child2 = cyclic_crossover(selected_parents[0], selected_parents[1])

        if random.random() < MUTATION_PROBABILITY:
            child1 = inversion_mutation(child1)
        if random.random() < MUTATION_PROBABILITY:
            child2 = inversion_mutation(child2)

        population[sorted_indices[-1]] = child1
        population[sorted_indices[-2]] = child2

    best_solution = population[sorted_indices[0]]
    return best_solution

# Main function
if __name__ == "__main__":
    optimal_route = genetic_algorithm()
    optimal_distance = total_distance(optimal_route)

    print("Optimal Route:", optimal_route)
    print("Optimal Distance:", optimal_distance)
